In [0]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, sum, lit, when
import pyspark.sql.functions as sf
from pyspark import pandas as ps
import pandas as pd
import numpy as np

In [0]:
dfspark = spark.read.table('workspace.1_bronze_db.students_bronze')
dfspark.printSchema()

In [0]:
display(dfspark.limit(5))

Data Cleaning

In [0]:
df = ps.DataFrame(dfspark.toPandas())

df.isna().any()

In [0]:
dfspark.filter(col("_rescued_data").isNotNull()).display()

In [0]:
income_median = dfspark.select(sf.round(sf.median("Family_Income"))).first()[0]
hours_avg = dfspark.select(sf.round(sf.avg("Study_Hours_per_Day"))).first()[0]
stress_avg = dfspark.select(sf.round(sf.avg("Stress_Index"))).first()[0]

dfspark = dfspark.na.fill({'Family_Income': income_median, 'Study_Hours_per_Day': hours_avg, 'Stress_Index': stress_avg})

In [0]:
dfspark = dfspark.drop("_rescued_data")

In [0]:
dfspark = dfspark.na.fill(value="Unknown", subset="Parental_Education")

In [0]:
display(dfspark)

Standarization

In [0]:
dfspark = dfspark.withColumn('Part_Time_Job', 
                            when(col("Part_Time_Job") == "Yes", 1)
                            .when(col("Part_Time_Job") == "No", 0))

dfspark = dfspark.withColumn("Internet_Access",
                            when(col("Internet_Access") == "Yes", 1)
                            .when(col("Internet_Access") == "No", 0))
display(dfspark)

In [0]:
dfspark = dfspark.withColumn("Semester",
                            when(col("Semester") == "Year 1", 1)
                            .when(col("Semester") == "Year 2", 2)
                            .when(col("Semester") == "Year 3", 3)
                            .when(col("Semester") == "Year 4", 4))
display(dfspark)

In [0]:
spark.sql(f'DROP TABLE IF EXISTS workspace.2_silver_db.students_silver')

In [0]:
dfspark.write.format("delta").mode("append").saveAsTable("workspace.2_silver_db.students_silver")